In [1]:
import nemo
import nemo.collections.asr as nemo_asr
import pytorch_lightning as pl
from omegaconf import DictConfig
import pathlib
import nemo.collections.asr as nemo_asr
import pytorch_lightning as pl
import os
import matplotlib.pyplot as plt
import re

import copy
import json
import os
import tempfile
from math import ceil
from typing import Dict, List, Optional, Union

import torch
from omegaconf import DictConfig, OmegaConf, open_dict
from pytorch_lightning import Trainer
from torch.utils.data import ChainDataset
from tqdm.auto import tqdm

from nemo.collections.asr.data import audio_to_text_dataset
from nemo.collections.asr.data.audio_to_text_dali import DALIOutputs
from nemo.collections.asr.losses.ctc import CTCLoss
from nemo.collections.asr.metrics.wer import WER
from nemo.collections.asr.models.asr_model import ASRModel, ExportableEncDecModel
from nemo.collections.asr.parts.mixins import ASRModuleMixin
from nemo.collections.asr.parts.preprocessing.perturb import process_augmentations
from nemo.core.classes.common import PretrainedModelInfo, typecheck
from nemo.core.neural_types import AudioSignal, LabelsType, LengthsType, LogprobsType, NeuralType, SpectrogramType
from nemo.utils import logging
from nemo.collections.tts.models import FastPitchHifiGanE2EModel

import IPython.display as ipd
import librosa
import librosa.display
import numpy as np
import torch
from matplotlib import pyplot as plt
%matplotlib inline

# Reduce logging messages for this notebook
from nemo.utils import logging
logging.setLevel(logging.ERROR)

from nemo.collections.tts.models import FastPitchModel
from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.helpers.helpers import regulate_len
from nemo.collections.tts.models import MelGanModel

import torch.nn.functional as F

try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel_yaml import YAML

from dataset import UnpairedAudioToBPEDataset

[NeMo W 2022-01-21 16:44:39 optimizers:47] Apex was not found. Using the lamb optimizer will error out.
[NeMo W 2022-01-21 16:44:39 nemo_logging:349] /home/boris/anaconda3/lib/python3.7/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[NeMo W 2022-01-21 16:44:39 nmse_clustering:54] Using eigen decomposition from scipy, upgrade torch to 1.9 or higher for faster clustering
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2022-01-21 16:44:39 nemo_logging:34

In [2]:
config_path = 'stt_en_citrinet_256_gamma_0_25'
config_name = 'model_config.yaml'
yaml = YAML(typ='safe')

with open(os.path.join(config_path, config_name)) as f:
    config = yaml.load(f)

config['tokenizer']['dir'] = 'citrinet_tokenizer/tokenizer_spe_unigram_v1024'
config['tokenizer']['type'] = 'bpe'

config['train_ds']['manifest_filepath']="../../datasets/LJSpeech-1.1/small_manifest.json"
# config['train_ds']['manifest_filepath']="../../datasets/an4/train_manifest.json"
config['train_ds']['batch_size'] = 1
config['train_ds']['num_workers'] = 12
config['train_ds']['pin_memory'] = True

config['validation_ds']['manifest_filepath']="../../datasets/LJSpeech-1.1/train_manifest_text.json"
# config['validation_ds']['manifest_filepath']="../../datasets/an4/test_manifest.json"
config['validation_ds']['batch_size'] = 1
config['validation_ds']['num_workers'] = 12
config['validation_ds']['pin_memory'] = True

config['spec_augment']['freq_masks'] = 0
config['spec_augment']['time_masks'] = 0
config['optim']['lr'] = 1e-1
config['optim']['name'] = 'novograd'
config['optim']['betas'] = [0.8, 0.25]
config['optim']['weight_decay'] = 0.001
config['optim']['sched']['warmup_steps']=1000
config['optim']['sched']['min_lr'] = 0.00001

config['tokenizer']['model_path'] = 'stt_en_citrinet_256_gamma_0_25/3d20ebb793c84a64a20c7ad26fc64d62_tokenizer.model'
config['tokenizer']['vocab_path'] = 'stt_en_citrinet_256_gamma_0_25/df5191f216004f10a268c44e90fdb63f_vocab.txt'
config['tokenizer']['spe_tokenizer_vocab'] = 'stt_en_citrinet_256_gamma_0_25/b774eaac83804907843607272fde21a4_tokenizer.vocab'

asr_model = nemo_asr.models.EncDecCTCModelBPE(cfg=DictConfig(config))

# asr_model.setup_optimization()

In [3]:
config_path = 'configs'
config_name = 'fastpitch_align.yaml'
yaml = YAML(typ='safe')

with open(os.path.join(config_path, config_name)) as f:
    config = yaml.load(f)

config['model']['train_ds']['manifest_filepath']="../../datasets/LJSpeech-1.1/small_manifest.json"
config['model']['train_ds']['batch_size'] = 1
config['model']['train_ds']['num_workers'] = 12
config['model']['train_ds']['pin_memory'] = True

config['model']['validation_ds']['manifest_filepath']="../../datasets/LJSpeech-1.1/train_manifest_text.json"
config['model']['validation_ds']['batch_size'] = 1
config['model']['validation_ds']['num_workers'] = 12
config['model']['validation_ds']['pin_memory'] = True

config['model']['optim']['lr'] = 1e-1
config['model']['optim']['name'] = 'novograd'
config['model']['optim']['betas'] = [0.8, 0.25]
config['model']['optim']['weight_decay'] = 0.001
config['model']['optim']['sched']['warmup_steps']=1000
config['model']['optim']['sched']['min_lr'] = 0.00001

spec_gen = FastPitchModel.from_config_dict(DictConfig(config['model']))

[NeMo E 2022-01-21 16:44:41 fastpitch:345] The train dataloader for FastPitchModel() has shuffle set to False!!!
[NeMo E 2022-01-21 16:44:41 vocabs:324] Torch distributed needs to be initialized before you initialized <nemo.collections.asr.data.vocabs.Phonemes object at 0x7f965c527250>. This class is prone to data access race conditions. Now downloading corpora from global rank 0. If other ranks pass this before rank 0, errors might result.
[NeMo E 2022-01-21 16:44:43 vocabs:324] Torch distributed needs to be initialized before you initialized <nemo.collections.asr.data.vocabs.Phonemes object at 0x7f964eab1450>. This class is prone to data access race conditions. Now downloading corpora from global rank 0. If other ranks pass this before rank 0, errors might result.
[NeMo E 2022-01-21 16:44:44 vocabs:324] Torch distributed needs to be initialized before you initialized <nemo.collections.asr.data.vocabs.Phonemes object at 0x7f9646dc0090>. This class is prone to data access race conditio

In [4]:
# config_path = 'conf/hifigan'
# config_name = 'hifigan22050.yaml'
# yaml = YAML(typ='safe')

# with open(os.path.join(config_path, config_name)) as f:
#     config = yaml.load(f)

# vocoder = HifiGanModel.from_config_dict(DictConfig(config))

In [5]:
config_path = 'conf'
config_name = 'model_config_melgan.yaml'
yaml = YAML(typ='safe')

with open(os.path.join(config_path, config_name)) as f:
    config = yaml.load(f)

vocoder = MelGanModel.from_config_dict(DictConfig(config))

In [6]:
asr_model.cuda()
spec_gen.cuda()
vocoder.train().cuda()

MelGanModel(
  (audio_to_melspec_precessor): FilterbankFeatures()
  (generator): MelGANGenerator(
    (melgan): Sequential(
      (0): ReflectionPad1d((3, 3))
      (1): Conv1d(80, 512, kernel_size=(7,), stride=(1,))
      (2): LeakyReLU(negative_slope=0.2)
      (3): ConvTranspose1d(512, 256, kernel_size=(16,), stride=(8,), padding=(4,))
      (4): ResidualStack(
        (stack): Sequential(
          (0): LeakyReLU(negative_slope=0.2)
          (1): ReflectionPad1d((1, 1))
          (2): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
          (3): LeakyReLU(negative_slope=0.2)
          (4): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
        )
        (skip_layer): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
      )
      (5): ResidualStack(
        (stack): Sequential(
          (0): LeakyReLU(negative_slope=0.2)
          (1): ReflectionPad1d((3, 3))
          (2): Conv1d(256, 256, kernel_size=(3,), stride=(1,), dilation=(3,))
          (3): LeakyReLU(negative_slope=0.2)

In [7]:
vocoder.setup_optimization()

(AdamW (
 Parameter Group 0
     amsgrad: False
     betas: [0.8, 0.99]
     eps: 1e-08
     lr: 0.1
     weight_decay: 0.01
 ),
 None)

In [8]:
def on_train_start():
    asr_model.configure_optimizers()
    spec_gen.configure_optimizers()
#     vocoder.configure_optimizers()
    asr_model.configure_callbacks()
    spec_gen.configure_callbacks()
    vocoder.configure_callbacks()

    asr_model.on_fit_start()
    spec_gen.on_fit_start()
    vocoder.on_fit_start()
    asr_model.setup("fit")
    spec_gen.setup("fit")
    vocoder.setup("fit")

    asr_model.on_pretrain_routine_start()
    spec_gen.on_pretrain_routine_start()
    vocoder.on_pretrain_routine_start()

    asr_model.on_pretrain_routine_end()
    spec_gen.on_pretrain_routine_end()
    vocoder.on_pretrain_routine_end()

    asr_model.on_train_start()
    spec_gen.on_train_start()
    vocoder.on_train_start()
    
def on_epoch_start():
    asr_model.on_epoch_start()
    spec_gen.on_epoch_start()
    vocoder.on_epoch_start()
    asr_model.on_train_epoch_start()
    spec_gen.on_train_epoch_start()
    vocoder.on_train_epoch_start()

In [9]:
def paired_asr_training_step(batch_idx, asr_batch, dataloader_idx=0):
    asr_model.on_train_batch_start(asr_batch, batch_idx, dataloader_idx)

    asr_model.on_before_batch_transfer(asr_batch, dataloader_idx)
    asr_model.transfer_batch_to_device(asr_batch, torch.device("cuda"), dataloader_idx)
    asr_model.on_after_batch_transfer(asr_batch, dataloader_idx)

    signal, signal_len, transcript, transcript_len = asr_batch
    
#     print(signal.shape, signal_len.shape)
    
    signal, signal_len, transcript, transcript_len = signal.cuda(), signal_len.cuda(), \
        transcript.cuda(), transcript_len.cuda()

    log_probs, encoded_len, predictions = asr_model.forward(input_signal=signal, input_signal_length=signal_len)

    loss = asr_model.loss(
        log_probs=log_probs, targets=transcript, input_lengths=encoded_len, target_lengths=transcript_len
    )

    asr_model.on_before_zero_grad(asr_model._optimizer)
    asr_model.optimizer_zero_grad(epoch, batch_idx, asr_model._optimizer, batch_idx)

    asr_model.on_before_backward(loss)
    asr_model.backward(loss, asr_model._optimizer, batch_idx)
    asr_model.on_after_backward()

    asr_model.on_before_optimizer_step(asr_model._optimizer, batch_idx)
    asr_model.optimizer_step(epoch, batch_idx, asr_model._optimizer, batch_idx)

    asr_model.on_train_batch_end(predictions, asr_batch, batch_idx, dataloader_idx)
    
    return loss

In [10]:
def paired_spec_gen_vocoder_step(batch_idx, tts_batch, dataloader_idx=0):
    spec_gen.on_train_batch_start(tts_batch, batch_idx, dataloader_idx)
        
    spec_gen.on_before_batch_transfer(tts_batch, dataloader_idx)
    spec_gen.transfer_batch_to_device(tts_batch, torch.device("cuda"), dataloader_idx)
    spec_gen.on_after_batch_transfer(tts_batch, dataloader_idx)

    audio, audio_lens, text, text_lens, attn_prior, pitch, speaker = tts_batch
    audio, audio_lens, text, text_lens, attn_prior, pitch, speaker = audio.cuda(), \
        audio_lens.cuda(), text.cuda(), text_lens.cuda(), attn_prior, pitch, speaker
    
    if pitch is not None:
        pitch = pitch.cuda()
        
    attn_prior, durs, speaker = None, None, None

    mels, spec_len = spec_gen.preprocessor(input_signal=audio, length=audio_lens)

    mels_pred, _, _, log_durs_pred, pitch_pred, attn_soft, attn_logprob, attn_hard, attn_hard_dur, pitch = \
    spec_gen.forward(
        text=text,
        durs=durs,
        pitch=pitch,
        speaker=speaker,
        pace=1.0,
        spec=mels,
        attn_prior=attn_prior,
        mel_lens=spec_len,
        input_lens=text_lens,
    )

    if durs is None:
        durs = attn_hard_dur

    mel_loss = spec_gen.mel_loss(spect_predicted=mels_pred, spect_tgt=mels)
    dur_loss = spec_gen.duration_loss(
        log_durs_predicted=log_durs_pred.cuda(), durs_tgt=durs.cuda(), len=text_lens.cuda())
    spec_gen_loss = mel_loss + dur_loss

    if pitch is not None:    
        pitch_loss = spec_gen.pitch_loss(pitch_predicted=pitch_pred.cuda(), pitch_tgt=pitch,
                                      len=text_lens.cuda())
    else:
        pitch_loss = 0
    spec_gen_loss += pitch_loss

    spec_gen.on_before_zero_grad(spec_gen._optimizer)
    spec_gen.optimizer_zero_grad(epoch, batch_idx, spec_gen._optimizer, batch_idx)

    spec_gen.on_before_backward(spec_gen_loss)
    spec_gen.backward(spec_gen_loss, spec_gen._optimizer, batch_idx)
    spec_gen.on_after_backward()

    spec_gen.on_before_optimizer_step(spec_gen._optimizer, batch_idx)
    spec_gen.optimizer_step(epoch, batch_idx, spec_gen._optimizer, batch_idx)

    spec_gen.on_train_batch_end(mels_pred, tts_batch, batch_idx, dataloader_idx)

    # vocoder

    vocoder.on_train_batch_start(tts_batch, batch_idx, dataloader_idx)

    vocoder.on_before_batch_transfer(tts_batch, dataloader_idx)
    vocoder.transfer_batch_to_device(tts_batch, torch.device("cuda"), dataloader_idx)
    vocoder.on_after_batch_transfer(tts_batch, dataloader_idx)
    
    audio_pred = vocoder.forward(spec=mels_pred)

    vocoder_loss = 0

    audio_ = audio_pred.squeeze(1)[:, :audio.shape[1]]

    sc_loss, mag_loss = vocoder.loss(x=audio_, y=audio)
    loss_feat = sum(sc_loss) + sum(mag_loss)
    loss_feat /= len(sc_loss)
    vocoder_loss += loss_feat

    vocoder.on_before_zero_grad(vocoder._optimizer)
    vocoder.optimizer_zero_grad(epoch, batch_idx, vocoder._optimizer, batch_idx)

    vocoder.on_before_backward(vocoder_loss)
#     try:
#         vocoder.backward(vocoder_loss, vocoder._optimizer, batch_idx)
#     except Exception:
#         pass
    vocoder.on_after_backward()

    vocoder.on_before_optimizer_step(vocoder._optimizer, batch_idx)
    vocoder.optimizer_step(epoch, batch_idx, vocoder._optimizer, batch_idx)

    vocoder.on_train_batch_end(audio_, tts_batch, batch_idx, dataloader_idx)
    
    loss = torch_mse_loss(audio_, audio)
    
    del audio_pred
    del sc_loss, mag_loss
    del audio, audio_
#     del spec_gen_loss, vocoder_loss
    del loss_feat
        
    return spec_gen_loss.cpu().detach(), vocoder_loss.cpu().detach()

In [11]:
def dual_transformation_asr_step(batch_idx, raw_signal_batch, dataloader_idx=0):
    signal, signal_len, _, _ = raw_signal_batch
    signal, signal_len = signal.cuda(), signal_len.cuda()

    log_probs, encoded_len, predictions = asr_model.forward(input_signal=signal, input_signal_length=signal_len)

    current_hypotheses = asr_model._wer.ctc_decoder_predictions_tensor(
        predictions, predictions_len=encoded_len, return_hypotheses=False,
    )
        
    transcript = spec_gen.parse('Hi')
        
    transcript_len = torch.tensor([len(transcript)])
        
    batch = (signal, signal_len, transcript, transcript_len, None, None, None)
    
    return paired_spec_gen_vocoder_step(batch_idx, batch)

In [12]:
def dual_transformation_tts_step(batch_idx, raw_text_batch, dataloader_idx=0):
    s, s_l, text, text_lens, _, _, _ = raw_text_batch
    
    s, s_l, text, text_lens = s.cuda(), s_l.cuda(), text.cuda(), text_lens.cuda()
    
#     target = asr_model._wer.ctc_decoder_predictions_tensor(
#         text, predictions_len=text_lens, return_hypotheses=False,
#     )
        
#     parsed = spec_gen.parse(target[0])
          
#     print(text)
        
    spec, _, durs_pred, _, pitch_pred, *_ = spec_gen.forward(text=text,
                                                        durs=None, pitch=None, speaker=None, pace=1.0)
        
#     print(spec, spec.shape)
        
    signal = vocoder.forward(spec=spec)
    signal_len = torch.tensor([len(signal[0])])
    
    signal = signal.squeeze(0)
    
    print('s', s.shape)
    print('s', signal.shape)
        
    return paired_asr_training_step(batch_idx, (signal, signal_len, text, text_lens))

In [13]:
def on_epoch_end():
    asr_model.on_train_epoch_end() 
    spec_gen.on_train_epoch_end()
    vocoder.on_train_epoch_end()
    
    asr_model.on_epoch_end()
    spec_gen.on_epoch_end()
    vocoder.on_epoch_end()
    
def on_train_end():
    asr_model.on_train_end()
    spec_gen.on_train_end()
    vocoder.on_train_end()

    asr_model.on_fit_end()
    spec_gen.on_fit_end()
    vocoder.on_fit_end()

    asr_model.teardown("fit")
    spec_gen.teardown("fit")
    vocoder.teardown("fit")

In [14]:
def flip_batch(batch):
    for i in range(len(batch)):
        try:
            for j in range(len(batch[i])):
                batch[i][j] = torch.flip(batch[i][j], dims=[0])
        except Exception:
            pass
    return batch

In [ ]:
max_epochs = 300

dataloader_idx = 0

on_train_start()

overall_loss = {'epoch': [], 'asr_model_loss': [], 'spec_gen_loss': [], 'vocoder_loss': []}

torch_mse_loss = torch.nn.MSELoss()

for epoch in range(max_epochs):
    epoch_loss = {'asr_model_loss': [], 'spec_gen_loss': [], 'vocoder_loss': []}
    
    on_epoch_start()
    
    for batch_idx, (asr_batch, tts_batch, raw_signal_batch, raw_text_batch) in \
        enumerate(zip(asr_model.train_dataloader(), spec_gen.train_dataloader(), 
                     asr_model.val_dataloader(), spec_gen.val_dataloader())):
        
#         asr_model_loss = dual_transformation_tts_step(batch_idx, raw_text_batch)


        asr_model_loss = paired_asr_training_step(batch_idx, asr_batch) 
        
        asr_model_loss += paired_asr_training_step(batch_idx, flip_batch(asr_batch)) 
        
        epoch_loss['asr_model_loss'].append(asr_model_loss)
        
        spec_gen_loss, vocoder_loss = paired_spec_gen_vocoder_step(batch_idx, tts_batch)
        _sgl, _vl = paired_spec_gen_vocoder_step(batch_idx, flip_batch(tts_batch))
        spec_gen_loss += _sgl
        vocoder_loss += _vl
        
        _sgl, _vl = dual_transformation_asr_step(batch_idx, raw_signal_batch)
        spec_gen_loss += _sgl
        vocoder_loss += _vl
        _sgl, _vl = dual_transformation_asr_step(batch_idx, flip_batch(raw_signal_batch))
        spec_gen_loss += _sgl
        vocoder_loss += _vl
                
        epoch_loss['spec_gen_loss'].append(spec_gen_loss)
        epoch_loss['vocoder_loss'].append(vocoder_loss)
        
        
        
#         print(asr_model_loss, spec_gen_loss, vocoder_loss)
        
#         del asr_model_loss, spec_gen_loss, vocoder_loss
        
    overall_loss['epoch'].append(epoch)
    overall_loss['asr_model_loss'].append(torch.mean(torch.tensor(epoch_loss['asr_model_loss'])))
    overall_loss['spec_gen_loss'].append(torch.mean(torch.tensor(epoch_loss['spec_gen_loss'])))
    overall_loss['vocoder_loss'].append(torch.mean(torch.tensor(epoch_loss['vocoder_loss'])))

    
    
    print(overall_loss)

    on_epoch_end()
    
on_train_end()

[NeMo E 2022-01-21 16:44:52 vocabs:324] Torch distributed needs to be initialized before you initialized <nemo.collections.asr.data.vocabs.Phonemes object at 0x7f95fa547990>. This class is prone to data access race conditions. Now downloading corpora from global rank 0. If other ranks pass this before rank 0, errors might result.


In [ ]:
# for batch_idx, batch in enumerate(UnpairedAudioToBPEDataset(manifest_filepath="../../datasets/LJSpeech-1.1/small_manifest.json",
#                     unpaired_text_manifest_filepath="../../datasets/LJSpeech-1.1/train_manifest_text.json",
#                     unpaired_signal_manifest_filepath="../../datasets/LJSpeech-1.1/train_manifest_signal.json",                   
#                     tokenizer=asr_model.tokenizer,
#                     sample_rate=22050)):    
    
#     signal, signal_len, transcript, transcript_len, raw_signal, raw_signal_len, raw_text, raw_text_len = batch
# #     signal, signal_len, transcript, transcript_len = signal.cuda(), signal_len.cuda(), \
# #         transcript.cuda(), transcript_len.cuda()
#     print(raw_text)
    
#     break

In [23]:
def val():
    torch.set_grad_enabled(False)
    asr_model.on_validation_start()
    asr_model.on_epoch_start()
    asr_model.on_validation_epoch_start()

    val_outs = []

    for batch_idx, val_batch in enumerate(asr_model.val_dataloader()):

        asr_model.on_validation_batch_start(val_batch, batch_idx, dataloader_idx)

        batch = asr_model.on_before_batch_transfer(val_batch, dataloader_idx)
        batch = asr_model.transfer_batch_to_device(batch, torch.device('cuda'), dataloader_idx)
        batch = asr_model.on_after_batch_transfer(batch, dataloader_idx)

        out = asr_model.validation_step(batch, batch_idx)

        val_loss, val_wer_num, val_wer_denom, val_wer = out
        
        asr_model.on_validation_batch_end(out, batch, batch_idx, dataloader_idx)
        val_outs.append(out)
        
    asr_model.on_epoch_end()
    asr_model.on_validation_end()

    torch.set_grad_enabled(True)
    
    return val_outs

val()

[{'val_loss': tensor(81.5763, device='cuda:0'),
  'val_wer_num': tensor(8., device='cuda:0'),
  'val_wer_denom': tensor(10., device='cuda:0'),
  'val_wer': tensor(0.8000, device='cuda:0')},
 {'val_loss': tensor(251.4926, device='cuda:0'),
  'val_wer_num': tensor(19., device='cuda:0'),
  'val_wer_denom': tensor(21., device='cuda:0'),
  'val_wer': tensor(0.9048, device='cuda:0')},
 {'val_loss': tensor(267.2849, device='cuda:0'),
  'val_wer_num': tensor(23., device='cuda:0'),
  'val_wer_denom': tensor(25., device='cuda:0'),
  'val_wer': tensor(0.9200, device='cuda:0')},
 {'val_loss': tensor(250.1237, device='cuda:0'),
  'val_wer_num': tensor(21., device='cuda:0'),
  'val_wer_denom': tensor(22., device='cuda:0'),
  'val_wer': tensor(0.9545, device='cuda:0')},
 {'val_loss': tensor(241.5255, device='cuda:0'),
  'val_wer_num': tensor(20., device='cuda:0'),
  'val_wer_denom': tensor(22., device='cuda:0'),
  'val_wer': tensor(0.9091, device='cuda:0')},
 {'val_loss': tensor(136.2797, device='cud

In [19]:
asr_model.transcribe(["../../datasets//LJSpeech-1.1/wavs/LJ015-0247.wav",
                     "../../datasets//LJSpeech-1.1/wavs/LJ028-0475.wav",
                     "../../datasets//LJSpeech-1.1/wavs/LJ013-0049.wav"])

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['⁇ ss ⁇', '⁇ ss ⁇', '⁇ ss ⁇']

In [8]:
# def fastpitch_training_step(self, audio, audio_lens, text, text_lens, durs, pitch, batch_idx=1):
#     mels, spec_len = self.preprocessor(input_signal=audio, length=audio_lens)

#     mels_pred, _, _, log_durs_pred, pitch_pred, attn_soft, attn_logprob, attn_hard, attn_hard_dur, pitch = self(
#         text=text,
#         durs=durs,
#         pitch=pitch,
#         speaker=None,
#         pace=1.0,
#         spec=mels if self.learn_alignment else None,
#         attn_prior=None,
#         mel_lens=spec_len,
#         input_lens=text_lens,
#     )
#     if durs is None:
#         durs = attn_hard_dur

#     mel_loss = self.mel_loss(spect_predicted=mels_pred, spect_tgt=mels)
#     dur_loss = self.duration_loss(log_durs_predicted=log_durs_pred, durs_tgt=durs, len=text_lens)
#     loss = mel_loss + dur_loss

#     pitch_loss = self.pitch_loss(pitch_predicted=pitch_pred, pitch_tgt=pitch, len=text_lens)
#     loss += pitch_loss

#     return loss

# def hifigan_training_step(self, batch, batch_idx, optimizer_idx):
#     # if in finetune mode the mels are pre-computed using a
#     # spectrogram generator
#     if self.input_as_mel:
#         audio, audio_len, audio_mel = batch
#     # else, we compute the mel using the ground truth audio
#     else:
#         audio, audio_len = batch
#         # mel as input for generator
#         audio_mel, _ = self.audio_to_melspec_precessor(audio, audio_len)

#     # mel as input for L1 mel loss
#     audio_trg_mel, _ = self.trg_melspec_fn(audio, audio_len)
#     audio = audio.unsqueeze(1)

#     audio_pred = self.generator(x=audio_mel)
#     audio_pred_mel, _ = self.trg_melspec_fn(audio_pred.squeeze(1), audio_len)

#     # train discriminator
# #     self.optim_d.zero_grad()
#     mpd_score_real, mpd_score_gen, _, _ = self.mpd(y=audio, y_hat=audio_pred.detach())
#     loss_disc_mpd, _, _ = self.discriminator_loss(
#         disc_real_outputs=mpd_score_real, disc_generated_outputs=mpd_score_gen
#     )
#     msd_score_real, msd_score_gen, _, _ = self.msd(y=audio, y_hat=audio_pred.detach())
#     loss_disc_msd, _, _ = self.discriminator_loss(
#         disc_real_outputs=msd_score_real, disc_generated_outputs=msd_score_gen
#     )
#     loss_d = loss_disc_msd + loss_disc_mpd
    
#     return loss_d
# #     self.manual_backward(loss_d)
# #     self.optim_d.step()

# #     # train generator
# #     self.optim_g.zero_grad()
# #     loss_mel = F.l1_loss(audio_pred_mel, audio_trg_mel)
# #     _, mpd_score_gen, fmap_mpd_real, fmap_mpd_gen = self.mpd(y=audio, y_hat=audio_pred)
# #     _, msd_score_gen, fmap_msd_real, fmap_msd_gen = self.msd(y=audio, y_hat=audio_pred)
# #     loss_fm_mpd = self.feature_loss(fmap_r=fmap_mpd_real, fmap_g=fmap_mpd_gen)
# #     loss_fm_msd = self.feature_loss(fmap_r=fmap_msd_real, fmap_g=fmap_msd_gen)
# #     loss_gen_mpd, _ = self.generator_loss(disc_outputs=mpd_score_gen)
# #     loss_gen_msd, _ = self.generator_loss(disc_outputs=msd_score_gen)
# #     loss_g = loss_gen_msd + loss_gen_mpd + loss_fm_msd + loss_fm_mpd + loss_mel * self.l1_factor
# #     self.manual_backward(loss_g)
# #     self.optim_g.step()

# def citrinet_training_step(self, batch, batch_nb):
#     signal, signal_len, transcript, transcript_len = batch
    
#     log_probs, encoded_len, predictions = self.forward(input_signal=signal, input_signal_length=signal_len)

#     loss_value = self.loss(
#         log_probs=log_probs, targets=transcript, input_lengths=encoded_len, target_lengths=transcript_len
#     )

#     return loss_value

# # for batch in asr_model.test_dataloader():
# #     signal, signal_len, transcript, transcript_len = batch
        
# #     target = asr_model._wer.ctc_decoder_predictions_tensor(
# #         transcript, predictions_len=transcript_len, return_hypotheses=False,
# #     )
    
# #     parsed = fastpitch.parse(target[0])
    
# #     spec, _, durs_pred, _, pitch_pred, *_ = fastpitch.eval()(text=parsed,
# #                                                              durs=None, pitch=None, speaker=None, pace=1.0)
        
# #     loss = fastpitch_training_step(fastpitch.cpu().train(), signal, signal_len, parsed,
# #                                    torch.tensor([len(parsed)]), durs_pred, pitch_pred, batch_idx=1).cpu()
    
# #     hifigan.input_as_mel = True
# #     loss_2 = hifigan_training_step(hifigan.cpu().train(), batch=[signal, signal_len, spec], batch_idx=1, optimizer_idx=1)
    
# #     loss_asr = citrinet_training_step(asr_model.cpu().train(), batch=batch, batch_nb=1)
    
# #     print(loss, loss_2, loss_asr)
# #     break

In [98]:
# # Dual Transformation: text -> predicted_speech -> predicted_text -> loss
# def dt_text_training_step(batch, batch_idx, optimizer_idx=None):
#     if optimizer_idx is None:
#         optimizer_idx = batch_idx
    
#     _, _, transcript, transcript_len = batch
        
#     target = asr_model._wer.ctc_decoder_predictions_tensor(
#         transcript, predictions_len=transcript_len, return_hypotheses=False,
#     )
    
#     parsed = fastpitch.parse(target[0]).cpu()
    
#     spec, _, durs_pred, _, pitch_pred, *_ = fastpitch.cpu().eval()(text=parsed,
#                                                              durs=None, pitch=None, speaker=None, pace=1.0)
        
#     fastpitch.eval()(text=parsed, durs=durs_pred, pitch=pitch_pred, speaker=None, pace=1.0)
    
#     signal = hifigan.cpu().eval().convert_spectrogram_to_audio(spec=spec).to('cpu')
#     signal_len = torch.tensor([len(signal[0])])
        
#     loss = citrinet_training_step(asr_model.cpu().train(),
#                                   batch=[signal, signal_len, transcript, transcript_len], batch_nb=batch_idx)

#     fastpitch_training_step(fastpitch.cpu().train(), signal, signal_len, parsed,
#                                    torch.tensor([len(parsed)]), durs_pred, pitch_pred, batch_idx=batch_idx).cpu()

#     hifigan.input_as_mel = True
#     hifigan_training_step(hifigan.cpu().train(),
#                                    batch=[signal, signal_len, spec], batch_idx=batch_idx, optimizer_idx=optimizer_idx)
    
#     return loss

In [94]:
# # Dual Transformation: speech -> predicted_text -> predicted_speech -> loss
# def dt_speech_training_step(batch, batch_idx, optimizer_idx=None):
#     if optimizer_idx is None:
#         optimizer_idx = batch_idx
        
#     signal, signal_len, _, _ = batch

#     logits, logits_len, greedy_predictions = asr_model.eval().cpu()(input_signal=signal,
#                                                                     input_signal_length=signal_len)
#     current_hypotheses = asr_model._wer.ctc_decoder_predictions_tensor(
#         greedy_predictions, predictions_len=logits_len
#     )
    
#     transcript = fastpitch.parse(current_hypotheses[0])
#     transcript_len = torch.tensor([len(transcript[0])]) 
        
#     spec, _, durs_pred, _, pitch_pred, *_ = fastpitch.eval()(text=transcript, durs=None, pitch=None, speaker=None, pace=1.0)
#     gt = train_batch[0]
# #     pred = hifigan.cpu().eval().convert_spectrogram_to_audio(spec=spec.cpu()).to('cpu')
                 
# #     citrinet_training_step(asr_model.cpu().train(),
# #                                   batch=[signal, signal_len, transcript, transcript_len], batch_nb=batch_idx)

# #     fastpitch_training_step(fastpitch.cpu().train(), signal, signal_len, transcript,
# #                                    transcript_len.cpu(), durs_pred, pitch_pred, batch_idx=batch_idx).cpu()

# #     hifigan.input_as_mel = True
# #     hifigan_training_step(hifigan.cpu().train(),
# #                                    batch=[signal, signal_len, spec], batch_idx=batch_idx, optimizer_idx=optimizer_idx)    
    
# #     return ((signal[0][:min(len(signal[0]), len(pred[0]))] - 
# #              pred[0][:min(len(signal[0]), len(pred[0]))].detach().numpy()) ** 2).sum()
    

In [24]:
# for step, train_batch in enumerate(asr_model.train_dataloader()):
#     loss = 0
    
#     loss += dt_text_training_step(train_batch, step)
# #     loss += dt_text_training_step(flip_batch(train_batch), step)
    
# #     loss += dt_speech_training_step(train_batch, step)
# #     loss += dt_text_training_step(flip_batch(train_batch), step)
    
#     ###
#     # Add loss for paired data
#     ###

#     ###
#     # Add loss for Denoising Auto-Encoder
#     ###

#     # print(step, loss)
    